<a href="https://colab.research.google.com/github/Valerie-Osawe/sql_queries/blob/main/project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Exploration With SQL

> Objectives
The objectives of this project is to use sqlite in python to explore and query the database.

About Database
> The database contains 6 tables which are customers, orders, employee, department, students and products.

Cumstomer table has 2 columns; 
- id 
- name

Orders table has 2 columns;
- id
- order-id

Employee table has 4 columns
- id 
- name
- salary
- dept_id

Department table has 2 columns
- id 
- dept_name

Products table has 3 columns 
- id
- store
- price

Students table has 6 olumns
- matric_no
- gender
- name
- maths
- english
- physics

In [21]:
import pandas as pd
import sqlite3

In [22]:
# connecting to the database
conn = sqlite3.connect('week_2.db')

# creating a cursor object
cur = conn.cursor()

In [23]:
# checking for table names
tabs = cur.execute('SELECT name FROM sqlite_master WHERE type="table";')
print(tabs.fetchall())

[('customers',), ('sqlite_sequence',), ('orders',), ('employee',), ('department',), ('students',), ('products',)]


## View tables

In [24]:
# view customers table
tab1 = pd.read_sql_query("""SELECT *
FROM customers""", conn)
tab1

,id,name
0,1,Mike
1,2,Afam
2,3,Deborah
3,4,Yusuf


In [25]:
# view orders table
tab2 = pd.read_sql_query("""SELECT *
FROM orders""", conn)
tab2

,id,order_id
0,1,5.0
1,2,NaN
2,3,NaN
3,4,3.0


In [26]:
tab3 = pd.read_sql_query("""SELECT *
FROM employee""", conn)
tab3

,id,name,salary,dept_id
0,1,Steve,32000,1
1,2,Bola,31000,1
2,3,Felix,33000,2
3,4,Babalola,30200,1
4,5,Okafor,25000,2
5,6,Adigun,30000,1
6,7,Gafar,41000,2
7,8,Amanda,53000,2


In [27]:
tab4 = pd.read_sql_query("""SELECT *
FROM department""", conn)
tab4

,id,dept_name
0,1,Sales
1,2,IT


In [28]:
tab5 = pd.read_sql_query("""SELECT *
FROM students""", conn)
tab5

,matric_no,name,gender,maths,english,physics
0,ux-012-013,Emeka,M,95,84,94
1,ux-012-014,Agnes,F,80,69,88
2,ux-012-015,Alimat,F,69,69,77
3,ux-012-017,Fred,M,28,57,39
4,ux-012-018,Ogechi,F,80,69,78
5,ux-012-019,Steve,M,61,69,72
6,ux-012-016,Esther,F,93,77,91
7,ux-012-020,Ahmed,M,87,94,85
8,ux-012-021,Samantha,F,52,69,48
9,ux-012-022,Philip,M,51,39,45


In [29]:
tab6 = pd.read_sql_query("""SELECT *
FROM products""", conn)
tab6

,id,store,price
0,0,store a,10
1,0,store c,12
2,1,store b,35
3,1,store a,33
4,1,store c,34
5,0,store b,11


## Data Exploration

## Qs 1: 

Who were the **customers** that did ***NOT*** place any orders. 

In [30]:
# using pandas to place the result of the query in a dataframe
df = pd.read_sql_query("""SELECT name
FROM customers
WHERE id NOT in (
  SELECT id
  FROM orders
WHERE order_id IS NOT NULL
);
""", conn)
df


,name
0,Afam
1,Deborah


## Qs 2:
What were the price of each product at every store.

In [31]:
df1 = pd.read_sql_query("""SELECT 
    id, 
    SUM(CASE WHEN store = 'store a' THEN price ELSE 0 END) AS store_a,
    SUM(CASE WHEN store = 'store b' THEN price ELSE 0 END) AS store_b,
    SUM(CASE WHEN store = 'store c' THEN price ELSE 0 END) AS store_c
FROM 
    products
GROUP BY 
    id;
""", conn)
df1

,id,store_a,store_b,store_c
0,0,10,11,12
1,1,33,35,34


## Qs 3: 
What are the salaries of each person in the different departments:


In [32]:
dept = pd.read_sql_query("""SELECT e.name, e.salary, d.dept_name
FROM employee e
INNER JOIN department d
ON e.dept_id = d.id
ORDER BY dept_name
;""", conn)
dept

,name,salary,dept_name
0,Felix,33000,IT
1,Okafor,25000,IT
2,Gafar,41000,IT
3,Amanda,53000,IT
4,Steve,32000,Sales
5,Bola,31000,Sales
6,Babalola,30200,Sales
7,Adigun,30000,Sales


## Qs 4
Who were the **highest earner** in **each department** and what were their salaries.

In [33]:
max_salary = pd.read_sql_query("""SELECT e.name, e.salary, d.dept_name
FROM employee e
INNER JOIN department d
ON e.dept_id = d.id
INNER JOIN (
  SELECT dept_id, MAX(salary) AS max_salary
  FROM employee
  GROUP BY dept_id
) max_salary ON e.dept_id = max_salary.dept_id AND e.salary = max_salary.max_salary
ORDER BY dept_name;""", conn)
max_salary

,name,salary,dept_name
0,Amanda,53000,IT
1,Steve,32000,Sales


## Qs 5
Show all the employees and their salaries in each department **except** the **lowest earner**. 

In [34]:
exclude_min = pd.read_sql_query("""SELECT e.name, e.salary, d.dept_name
FROM employee e
INNER JOIN department d
ON e.dept_id = d.id
INNER JOIN (
  SELECT dept_id, MIN(salary) AS min_salary
  FROM employee
  GROUP BY dept_id
) min_salary ON e.dept_id = min_salary.dept_id AND e.salary > min_salary.min_salary
ORDER BY dept_name;""", conn)
exclude_min

,name,salary,dept_name
0,Amanda,53000,IT
1,Felix,33000,IT
2,Gafar,41000,IT
3,Babalola,30200,Sales
4,Bola,31000,Sales
5,Steve,32000,Sales


### Qs 6
What is the percentage score of each student in descending order.

In [35]:
# create a new column for percentage score
percent_avg = pd.read_sql_query(""" SELECT *, ROUND(((maths + english + physics) / 3), 2) AS percentage_score
FROM students
GROUP BY name
ORDER BY percentage_score DESC;
""", conn)
percent_avg

,matric_no,name,gender,maths,english,physics,percentage_score
0,ux-012-013,Emeka,M,95,84,94,91.0
1,ux-012-020,Ahmed,M,87,94,85,88.0
2,ux-012-016,Esther,F,93,77,91,87.0
3,ux-012-014,Agnes,F,80,69,88,79.0
4,ux-012-018,Ogechi,F,80,69,78,75.0
5,ux-012-023,Dave,M,77,69,70,72.0
6,ux-012-015,Alimat,F,69,69,77,71.0
7,ux-012-019,Steve,M,61,69,72,67.0
8,ux-012-024,Seyi,M,54,83,63,66.0
9,ux-012-021,Samantha,F,52,69,48,56.0


## Qs 7
Who are the students that scored more than the mean of the percentage_score.

In [36]:
# create the percentage score column, calculate the mean percentage score, select students with percentage score higher than mean
mean_percent= pd.read_sql_query("""SELECT *
FROM (
  SELECT *, ROUND(((maths + english + physics) / 3), 2) AS percentage_score
  FROM students
  GROUP BY name
)
WHERE percentage_score > (
  SELECT ROUND(AVG(percentage_score), 2)
  FROM (
    SELECT *, ROUND(((maths + english + physics) / 3), 2) AS percentage_score
    FROM students
    GROUP BY name
  ) AS subquery
) 
ORDER BY percentage_score DESC;""", conn)
mean_percent

,matric_no,name,gender,maths,english,physics,percentage_score
0,ux-012-013,Emeka,M,95,84,94,91.0
1,ux-012-020,Ahmed,M,87,94,85,88.0
2,ux-012-016,Esther,F,93,77,91,87.0
3,ux-012-014,Agnes,F,80,69,88,79.0
4,ux-012-018,Ogechi,F,80,69,78,75.0
5,ux-012-023,Dave,M,77,69,70,72.0
6,ux-012-015,Alimat,F,69,69,77,71.0


In [37]:
# close connection
conn.close()